**Hashmap From Scratch**
Design HashMap: Design a HashMap without using any built-in hash table libraries.
NOTE: All keys and values will be in the range of [0, 1000000].
      The number of operations will be in the range of [1, 10000].

We're given the max number of inputs (maxSize = 10000) so I'm inclined twoards a single array
using linear probing. A TON of wasted space on smaller inputs, but generally better
performance than separate chaining. 
So we need an array of size 10000. try key % maxSize, if in use, linear probe for slot
keep track of size so that if size == maxSize, can't put new data in.
must keep track of where data has been deleted so that if key x needed linear probe for insertion,
then the item in the previous space was removed, looking for x will continue beyond the first slot
so when you delete, leave -1. 

Average runtime should be O(1) for each operation, getting worse (trending towards O(n)) as the
map gets fuller. O(N) space, where N is the max size of the array, not the number of inputs

In [ ]:
class MyHashMap:
    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.maxSize = 10000
        self.table = [None] * self.maxSize
        self.size = 0
        
    def getIndex(self, key: int) -> int:
        """
        returns index for given key, or -1 if not there and table is full
        keep track of checks: if you've checked every slot, return -1
        """
        index = key % self.maxSize
        checks = 0
        while self.table[index]:
            if self.table[index] != -1 and self.table[index][0] == key:
                return index
            if checks == self.maxSize:
                return -1
            index  = (index + 1) % self.maxSize
            checks += 1
        return -1

    def put(self, key: int, value: int) -> None:
        """
        value will always be non-negative.
        """
        if self.size == self.maxSize:
            return
        index = key % self.maxSize
        while self.table[index] and self.table[index] != -1 and self.table[index][0] != key:
            index  = (index + 1) % self.maxSize
        self.table[index] = (key, value)
        self.size += 1
        
    def get(self, key: int) -> int:
        """
        Returns the value to which the specified key is mapped, or -1 if this map contains no mapping for the key
        """
        index = self.getIndex(key)
        if index == -1:
            return -1
        return self.table[index][1]
        

    def remove(self, key: int) -> None:
        """
        Removes the mapping of the specified value key if this map contains a mapping for the key
        """
        if self.size == 0:
            return
        index = self.getIndex(key)
        if index != -1:
            self.table[index] = -1
            self.size -= 1

**Find First Unique Character**
Given a string, find the first non-repeating character in it and return it's index. If it doesn't
exist, return -1.
    
Frequency counts should always be dicts! but here I want to do this in one pass, so the dict
should track not only frequncy of each letter, but also it's first appearance in the string.that
way once we have the frequencies, we can loop through the keys in the dict instead of the letters
in the string, which will usually have duplicates and therefore be a longer loop.
~~~~
                  # index, freq
s = "leetcode" {'l': [0, 1], 'e': [1, 1X 2], 't': [3, 1]} etc ==>
{'l': [0, 1], 'e': [1, 3], 't': [3, 1], 'c': [4, 1], 'o': [5, 1], 'd': [6, 1]}
now we can loop through the dict, keeping track of the min index of letters that appear once.
~~~~
we need to go through each letter at least once, so O(n) minimum runtime. Ours loops twice so 
O(n + n) ==> O(n) and upper bound of O(n) space (in the case of every letter being unique, hits
the upper bound, but always affected directly by n regardless). 

In [ ]:
class FirstUniqChar:
    def firstUniqChar(self, s: str) -> int:
        import math
        chars = {}
        lenS = len(s)
        for i in range(lenS):
            if s[i] in chars:
                chars[s[i]][1] += 1
            else:
                chars[s[i]] = [i, 1]
        minI = math.inf
        for char in chars:
            if chars[char][1] == 1 and chars[char][0] < minI:
                minI = chars[char][0]
        if minI == math.inf:
            return -1
        return minI

**Intersection**
Given two arrays, write a function to compute their intersection.
ex: nums1 = [1,2,2,1], nums2 = [2,2] ==> Output: [2]

Is it cheating to use set intersection? if not, i think the fastest possible answer is
return list(set(nums1).intersection(nums2)) (runtime-wise, not just code length)

But it probably is cheating. I'd like to use a set to keep track regardless, but if that's
also cheating, a dictionary that, instead of keeping track of frequencies, will only be added to
if num not in dict: ... but legit that's a set so...

either way, we want to get the smaller list because that already limits the outcome, and loop
through it. for each num, if that num is in the bigger list, add it to the set.

this is worst case O(n * m) when there is no overlap between the lists. but this is dumb, just use
set intersection (which has the same worst case runtime). space is O(n) for the overlap set

In [ ]:
class Intersection:
    def intersection(self, nums1: List[int], nums2: List[int]) -> List[int]:
        overlap = set()
        if len(nums2) < len(nums1):
            nums1, nums2 = nums2, nums1
            
        for num in nums1:
            if num in nums2:
                overlap.add(num)
        return list(overlap)
    
        # but don't do this, just do return list(set(nums1).intersection(nums2))

**LRU Cache**
Design and implement a data structure for Least Recently Used (LRU) cache. It should support the
following operations:
get(key) - Get the value (will always be positive) of the key if the key exists in the cache,
otherwise return -1.
put(key, value) - Set or insert the value if the key is not already present. When the cache
reached its capacity, it should invalidate the least recently used item before inserting a new
item.
The cache is initialized with a positive capacity.
Could you do both operations in O(1) time complexity?

ok so we need to make a hashmap again for O(1) runtime in both methods. but now each key must
track both a value and its last access. so almost a combination of making your own hashmap and
finding the first unique character in a string. so for putting, you must keep track of if slot is
the key you're looking for, if slot is empty, and the index of the least recently used element.
so that if you get through all slots and don't find the key or an empty slot, you don't have
to search again for the index to write over. this is still O(N) where N = the size of the array,
but not n the number of inputs, and still average case O(1). 

I initially tried to implement this the same way as building a hashmap, with an array the size of
the capacity and then tracking the lru element and lruIndex to overwrite when full. However this
led to a `time limit exceeded` error from leetcode. I had felt like using a dictionary would be 
cheating (in the same way it would have been when building your own hashmap), but it turns out
it's the only way. Ordered Dicts are perfectly designed for this problem, so it ended up being
quite simple.

In [ ]:
class LRUCache:
    def __init__(self, capacity: int):
        from collections import OrderedDict
        self.cache = OrderedDict()
        self.cap = capacity
        self.used = 0

    def get(self, key: int) -> int:
        if key in self.cache:
            val = self.cache.pop(key)
            self.cache[key] = val
            return val
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self.cache.pop(key)
            self.cache[key] = value
        elif self.used < self.cap:
            self.cache[key] = value
            self.used += 1
        else:
            self.cache.popitem(last=False)
            self.cache[key] = value


**Alien Dictionary**
In an alien language, surprisingly they also use english lowercase letters, but possibly in a
different order. The order of the alphabet is some permutation of lowercase letters.
Given a sequence of words written in the alien language, and the order of the alphabet, return
true if and only if the given words are sorted lexicographicaly in this alien language.

first off, we're going to have to continuously check the alphabet for ordering, so we should 
minimize the cost of that by turning that into a dictionary (O(n) to create once, but O(1) to
check in each comparison). so now for actually checking the order. we need to compare each word
to the one next to it. if at any point something is in the wrong order, return false. if it's
clearly in the right order (word1[x] < word2[x], NOT if they're equal), we can break out of the
loop cuz they're good to go. but how to check if they're the same? i want to keep a boolean flag
of same = True, and then if we break cuz it's correct, set same = False first. if same and
len(word1) > len(word2) we can also return False. if we get all the way through, return true.
~~~~
ex ["wor", "word", "world"] "worldabcefghijkmnpqstuvxyz"
     +++    +++  # same is true but len(word1) < len(word2) so keep going
            +++X    +++X # d > l in order, return false
~~~~
the runtime is O(n) to make the hashmap  + O(n) for the outer loop * O(m) where m = len(word) so
O(n + n*m) --> O(n*m).
space is O(x) where x = len(order) for the dict, so O(n). 

In [ ]:
class AlienDict:
    def isAlienSorted(self, words: List[str], order: str) -> bool:
        n  = len(words)
        if n <= 1: return True
        alph = {order[i]: i for i in range(len(order))}
        for i in range(n - 1):
            word1, word2 = words[i], words[i + 1]
            m = min(len(word1), len(word2))
            same = True
            for j in range(m):
                if alph[word1[j]] > alph[word2[j]]:
                    return False
                if alph[word1[j]] < alph[word2[j]]:
                    same = False
                    break
            if same and len(word1) > len(word2): return False
        return True